# Multi-agents systems

* coordinator agent/project manager
    * delegates to other agents
    * knows how to coordinate agents

    * scheduler agent
        * returns free slots
        * tools
            * calendar
            * invite
    * agenda agent
        * returns agenda
        * tools
            * agenda
    
* from 1 agent, 4 tools, 3 system instructions => 3 agents, 1-2 tools, 1 system instruction
* more transparent, reason, not overloading context windows etc


### Building a Meeting Scheduling System with Specialized Agents

Let’s look at what a project management system where two agents work together to identify and schedule necessary meetings might look like. The project management agent decides when meetings are needed, while a scheduling specialist handles the logistics of actually arranging them.

#### Tools for the Scheduling Specialist

First, let’s look at the tools available to our scheduling specialist. This agent needs to interact with calendars and create invites:

```python
@register_tool()
def check_availability(
    action_context: ActionContext,
    attendees: List[str],
    start_date: str,
    end_date: str,
    duration_minutes: int,
    _calendar_api_key: str
) -> List[Dict]:
    """Find available time slots for all attendees."""
    return calendar_service.find_available_slots(...)
```

```python
@register_tool()
def create_calendar_invite(
    action_context: ActionContext,
    title: str,
    description: str,
    start_time: str,
    duration_minutes: int,
    attendees: List[str],
    _calendar_api_key: str
) -> Dict:
    """Create and send a calendar invitation."""
    return calendar_service.create_event(...)
```

#### Scheduler Agent

The scheduling specialist is focused entirely on finding times and creating meetings:

```python
scheduler_agent = Agent(
    goals=[
        Goal(
            name="schedule_meetings",
            description="""Schedule meetings efficiently by:
            1. Finding times that work for all attendees
            2. Creating and sending calendar invites
            3. Handling any scheduling conflicts"""
        )
    ],
    ...
)
```

#### Project Management Agent Tools

Now let’s look at our project management agent. This agent focuses on project status and deciding when meetings are needed:

```python
@register_tool()
def get_project_status(
    action_context: ActionContext,
    project_id: str,
    _project_api_token: str
) -> Dict:
    """Retrieve current project status information."""
    return project_service.get_status(...)
```

```python
@register_tool()
def update_project_log(
    action_context: ActionContext,
    entry_type: str,
    description: str,
    _project_api_token: str
) -> Dict:
    """Record an update in the project log."""
    return project_service.log_update(...)
```

```python
@register_tool()
def call_agent(
    action_context: ActionContext,
    agent_name: str,
    task: str
) -> Dict:
    """Delegate to a specialist agent."""
    # Implementation as shown in previous tutorial
```

#### Project Manager Agent

The project management agent uses these tools to monitor progress and arrange meetings when needed:

```python
project_manager = Agent(
    goals=[
        Goal(
            name="project_oversight",
            description="""Manage project progress by:
            1. Getting the current project status
            2. Identifying when meetings are needed if there are issues in the project status log
            3. Delegating meeting scheduling to the "scheduler_agent" to arrange the meeting
            4. Recording project updates and decisions"""
        )
    ],
    ...
)
```

#### Division of Responsibilities

* The project manager understands project status and when meetings are needed.
* The scheduler excels at finding available times and managing calendar logistics.
* The `call_agent` tool allows seamless collaboration between them.

#### The `call_agent` Tool

* **Memory Isolation:** Each invoked agent gets its own memory instance, preventing confusion between different agents’ conversation histories.
* **Context Management:** Carefully control what context properties are passed to the invoked agent, preventing infinite recursion while ensuring necessary resources are available.
* **Result Handling:** Extract the final memory item as the result, providing a clean way to return information to the calling agent.

#### Registering Agents

To make this system work, we need to register our agents in the registry:

```python
class AgentRegistry:
    def __init__(self):
        self.agents = {}
        
    def register_agent(self, name: str, run_function: callable):
        """Register an agent's run function."""
        self.agents[name] = run_function
        
    def get_agent(self, name: str) -> callable:
        """Get an agent's run function by name."""
        return self.agents.get(name)

# When setting up the system
registry = AgentRegistry()
registry.register_agent("scheduler_agent", scheduler_agent.run)

# Include registry in action context
action_context = ActionContext({
    'agent_registry': registry,
    # Other shared resources...
})
```

#### ActionContext and Delegation Flow

The `ActionContext` provides a clean way to make the agent registry available to the `call_agent` tool without exposing it directly to all tools. When an agent needs to delegate a task, it simply uses the tool like any other, and the environment system handles the details of finding and invoking the right agent.

This architecture allows us to build complex multi-agent systems where each agent maintains its specialization while being able to collaborate with other agents when needed. The memory isolation ensures that each agent works with a clean context, while the result extraction provides a standard way to pass information back to the calling agent.


## Agent interaction and memory

* manager -> intern process
    * task description -> solution
    * quality dependent on the task description handed by manager, context important

* manager -> intern stream of tasks
    * gathering information along the way, storing them into a log
    * share log with the intern/agent
        * process task based on the important info in the log
        * log their thought process
    * manager can access the output and thought process
    * downside here is the complexity that might not be useful (reason for using more focused agents in the first place)




### Memory Interaction Patterns in Multi-Agent Systems

When agents work together, how they share and manage memory dramatically affects their collaboration. Let’s explore different patterns for memory interaction between agents, understanding when each pattern is most useful and how to implement it.

#### Message Passing: The Basic Pattern

The simplest form of agent interaction is message passing, where one agent sends a request and receives a response. This is like sending an email to a colleague — they get your message, do some work, and send back their results. You don’t see how they arrived at their answer; you just get their final response.

##### Implementation

```python
@register_tool()
def call_agent(action_context: ActionContext, 
               agent_name: str, 
               task: str) -> dict:
    """Basic message passing between agents."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)
    
    # Create fresh memory for the invoked agent
    invoked_memory = Memory()
    
    # Run agent and get result
    result_memory = agent_run(
        user_input=task,
        memory=invoked_memory
    )
    
    # Return only the final memory item
    return {
        "result": result_memory.items[-1].get("content", "No result")
    }
```

This pattern works well when the first agent only needs the final answer, not the reasoning process. For example, if a project manager agent asks a scheduling agent to find a meeting time, it might only need to know when the meeting was scheduled, not how the time was chosen.

#### Memory Reflection: Learning from the Process

Sometimes we want the first agent to understand how the second agent reached its conclusion. This is like asking a colleague to not just give you their answer, but to explain their thought process. We can achieve this by copying all of the second agent’s memories back to the first agent.

##### Implementation

```python
@register_tool()
def call_agent_with_reflection(action_context: ActionContext, 
                             agent_name: str, 
                             task: str) -> dict:
    """Call agent and receive their full thought process."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)
    
    # Create fresh memory for invoked agent
    invoked_memory = Memory()
    
    # Run agent
    result_memory = agent_run(
        user_input=task,
        memory=invoked_memory
    )
    
    # Get the caller's memory
    caller_memory = action_context.get_memory()
    
    # Add all memories from invoked agent to caller
    # although we could leave off the last memory to
    # avoid duplication
    for memory_item in result_memory.items:
        caller_memory.add_memory({
            "type": f"{agent_name}_thought",  # Mark source of memory
            "content": memory_item["content"]
        })
    
    return {
        "result": result_memory.items[-1].get("content", "No result"),
        "memories_added": len(result_memory.items)
    }
```

This pattern is valuable when the first agent needs to understand the reasoning process. For instance, if a research coordinator agent asks a data analysis agent to study some results, seeing the analysis process helps the coordinator better understand and use the conclusions.

#### Memory Handoff: Continuing the Conversation

Sometimes we want the second agent to pick up where the first agent left off, with full context of what’s happened so far. This is like having a colleague step in to take over a project — they need to know everything that’s happened up to that point.

##### Implementation

```python
@register_tool()
def hand_off_to_agent(action_context: ActionContext, 
                      agent_name: str, 
                      task: str) -> dict:
    """Transfer control to another agent with shared memory."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)
    
    # Get the current memory to hand off
    current_memory = action_context.get_memory()
    
    # Run agent with existing memory
    result_memory = agent_run(
        user_input=task,
        memory=current_memory  # Pass the existing memory
    )
    
    return {
        "result": result_memory.items[-1].get("content", "No result"),
        "memory_id": id(result_memory)
    }
```

This pattern is useful for complex tasks where context is crucial. For example, if a customer service agent hands off to a technical support agent, the technical agent needs to know the full history of the customer’s issue.


## Focusing agents attention

* limited task size, limited context size
* asymmetry -> a lot of tokens in (ie memory), not a lot coming out (task description)
* memory -> set of messages; agent pre-select relevant part of memory and pass them to other agents in task description
    * one agent identify relevant messages and output ids of the relevant messages (other agent can look it up), and add it to the task


### Selective Memory Sharing: Using LLM Understanding for Context Selection

Sometimes we want an agent to intelligently choose which parts of its memory to share with another agent. Instead of using rigid rules, we can leverage the LLM’s understanding of context to select the most relevant memories for the task at hand.

#### Implementation (LLM-guided memory selection)

```python
@register_tool(description="Delegate a task to another agent with selected context")
def call_agent_with_selected_context(action_context: ActionContext,
                                   agent_name: str,
                                   task: str) -> dict:
    """Call agent with LLM-selected relevant memories."""
    agent_registry = action_context.get_agent_registry()
    agent_run = agent_registry.get_agent(agent_name)
    
    # Get current memory and add IDs
    current_memory = action_context.get_memory()
    memory_with_ids = []
    for idx, item in enumerate(current_memory.items):
        memory_with_ids.append({
            **item,
            "memory_id": f"mem_{idx}"
        })
    
    # Create schema for memory selection
    selection_schema = {
        "type": "object",
        "properties": {
            "selected_memories": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "ID of a memory to include"
                }
            },
            "reasoning": {
                "type": "string",
                "description": "Explanation of why these memories were selected"
            }
        },
        "required": ["selected_memories", "reasoning"]
    }
    
    # Format memories for LLM review
    memory_text = "\n".join([
        f"Memory {m['memory_id']}: {m['content']}" 
        for m in memory_with_ids
    ])
    
    # Ask LLM to select relevant memories
    selection_prompt = f'''Review these memories and select the ones relevant for this task:

Task: {task}

Available Memories:
{memory_text}

Select memories that provide important context or information for this specific task.
Explain your selection process.'''
    
    # Self-prompting magic to find the most relevant memories
    selection = prompt_llm_for_json(
        action_context=action_context,
        schema=selection_schema,
        prompt=selection_prompt
    )
    
    # Create filtered memory from selection
    filtered_memory = Memory()
    selected_ids = set(selection["selected_memories"])
    for item in memory_with_ids:
        if item["memory_id"] in selected_ids:
            # Remove the temporary memory_id before adding
            item_copy = item.copy()
            del item_copy["memory_id"]
            filtered_memory.add_memory(item_copy)
    
    # Run the agent with selected memories
    result_memory = agent_run(
        user_input=task,
        memory=filtered_memory
    )
    
    # Add results and selection reasoning to original memory
    current_memory.add_memory({
        "type": "system",
        "content": f"Memory selection reasoning: {selection['reasoning']}"
    })
    
    for memory_item in result_memory.items:
        current_memory.add_memory(memory_item)
    
    return {
        "result": result_memory.items[-1].get("content", "No result"),
        "shared_memories": len(filtered_memory.items),
        "selection_reasoning": selection["reasoning"]
    }
```

#### Why this helps

* Each memory gets a unique ID for precise reference.
* The LLM reviews all memories in the context of the task and selects only what’s relevant.
* Structured JSON captures both the selection and the reasoning, increasing transparency.
* The originating agent retains a record of what was shared and why.

#### Example

```python
# Example memory contents:
memories = [
    {"type": "user", "content": "We need to build a new reporting dashboard"},
    {"type": "assistant", "content": "Initial cost estimate: $50,000"},
    {"type": "user", "content": "That seems high"},
    {"type": "assistant", "content": "Breakdown: $20k development, $15k design..."},
    {"type": "system", "content": "Project deadline updated to Q3"},
    {"type": "user", "content": "Can we reduce the cost?"}
]
```

```json
{
  "selected_memories": ["mem_1", "mem_3", "mem_5"],
  "reasoning": "Selected memories containing cost information and the request for cost reduction, excluding project timeline and general discussion as they're not directly relevant to the budget review task."
}
```

#### Advantages over rule-based filtering

* Understands context and implications rather than only pattern matching.
* Preserves human-readable reasoning for auditability.
* Adapts across tasks without code changes.
* Keeps sensitive or irrelevant history out of scope for downstream agents.

#### When to use

* You need targeted context for a specialist agent (e.g., budget review) without overwhelming it with full history.
* You want traceability of what was shared and the rationale.
* You must reduce noise while maintaining performance and relevance.

#### Recap of the four memory-sharing patterns

* **Message passing:** simple request/response with only final answer.
* **Memory reflection:** caller learns from the callee’s full process.
* **Memory handoff:** callee continues with the caller’s full context.
* **Selective memory sharing (this pattern):** provide only the most relevant context chosen by an LLM.

#### Choosing the right pattern — key questions

* How much context does the second agent need?
* Does the first agent need to understand the second agent’s process?
* Should the conversation history be preserved or minimized?
* Is there sensitive or irrelevant information to filter?


## Providing agentic AI information about the world

* treat LLM as an intern with general knowledge about the world, not about specific context
* we need to make sure that apart from the context, up-to-date information is fed into the system